In [43]:
# Standard python library
import configparser
import sys
import os
import argparse
from collections import namedtuple

# Local resources
from gref4hsi.scripts import georeference
from gref4hsi.scripts import orthorectification
from gref4hsi.utils import parsing_utils, specim_parsing_utils
from gref4hsi.scripts import visualize
from gref4hsi.utils.config_utils import prepend_data_dir_to_relative_paths
from gref4hsi.utils.config_utils import customize_config

# Third party
import numpy as np

In [44]:
# From seabeepy/notebooks/flight_runner
import datetime as dt
import os
from pathlib import Path

from config import SETTINGS
from pyodm import Node
from subprocess import CalledProcessError

import seabeepy as sb


In [45]:


# Login to MinIO
minio_client = sb.storage.minio_login(
    user=SETTINGS.MINIO_ACCESS_ID, password=SETTINGS.MINIO_SECRET_KEY
)



In [46]:
# Parent directories containing flight folders to process
base_dirs = [
    r"/home/notebook/shared-seabee-ns9879k/ntnu",
]

# Directory for temporary files
temp_dir = r"/home/notebook/cogs"

In [47]:
# Run info
run_date = dt.datetime.today()
print(f"Processing started: {run_date}")

Processing started: 2024-03-13 10:24:29.527411


In [48]:
# Get all potential mission folders for NodeODM
# (i.e. folders containing a 'config.seabee.yaml' and an 'capture' subdirectory, but NOT an 'orthophoto' directory)
mission_list = [
    f.parent
    for base_dir in base_dirs
    for f in Path(base_dir).rglob("config.seabee.yaml")
    if sb.ortho.check_subdir_exists(f.parent, "capture")
    and not sb.ortho.check_subdir_exists(f.parent, "processed")
]

In [49]:
mission_list

[PosixPath('/home/notebook/shared-seabee-ns9879k/ntnu/2024/slettvik_seaside-straumen_202402191358_ntnu_hyperspectral_74m'),
 PosixPath('/home/notebook/shared-seabee-ns9879k/ntnu/2024/slettvik_hopavaagen_202402191311_ntnu_hyperspectral_74m'),
 PosixPath('/home/notebook/shared-seabee-ns9879k/ntnu/2024/slettvik_hopavaagen_202402191253_ntnu_hyperspectral_74m')]

In [57]:
# Establish the ancillary data paths, copy to local work space and 
from pathlib import Path

geoid_path_minio = Path('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/geoids/no_kv_HREF2018A_NN2000_EUREF89.tif')
config_template_path_minio = Path('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/configuration_specim.ini')
lab_calibration_path_minio = Path('/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/Lab_Calibrations')

geoid_path = os.path.join(temp_dir, "no_kv_HREF2018A_NN2000_EUREF89.tif")
try:
    shutil.copyfile(geoid_path_minio, geoid_path)
except FileExistsError:
    pass

config_template_path = os.path.join(temp_dir, "config_template_path_specim.ini")
try:
    shutil.copyfile(config_template_path_minio, config_template_path)
except FileExistsError:
    pass

lab_calibration_path = os.path.join(temp_dir, "lab-calibration")
try:
    shutil.copytree(lab_calibration_path_minio, lab_calibration_path)
except FileExistsError:
    pass

In [60]:
# Establish the ancillary data paths
import yaml
import specim_process
import shutil

# First locate mission and copy to local 
specim_mission_folder_minio = mission_list[0]
specim_mission_folder = os.path.join(temp_dir, "mission-data")
try:
    shutil.copytree(specim_mission_folder_minio, specim_mission_folder)
except FileExistsError:
    pass

config_yaml = os.path.join(specim_mission_folder, "config.seabee.yaml")

src_folder = '/home/notebook/shared-seabee-ns9879k/ntnu/specim_processing_data/Lab_Calibrations'
dst_folder = temp_dir + 'Lab_calibration'



specim_process.main(str(config_yaml), str(specim_mission_folder), geoid_path, config_template_path, lab_calibration_path)
    

    



DEM folder does not exist so Geoid is used as terrain instead


/opt/conda/lib/python3.10/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


eul_zyx
position_ecef
nav_timestamp
radiance_cube
t_exp_ms
hsi_timestamps
view_angles
wavelengths
fwhm
dark_frame
radiometric_frame
eul_zyx
position_ecef
nav_timestamp
radiance_cube
t_exp_ms
hsi_timestamps
view_angles
wavelengths
fwhm
dark_frame
radiometric_frame
eul_zyx
position_ecef
nav_timestamp
radiance_cube
t_exp_ms
hsi_timestamps
view_angles
wavelengths
fwhm
dark_frame
radiometric_frame
eul_zyx
position_ecef
nav_timestamp
radiance_cube
t_exp_ms
hsi_timestamps
view_angles
wavelengths
fwhm
dark_frame
radiometric_frame
eul_zyx
position_ecef
nav_timestamp
radiance_cube
t_exp_ms
hsi_timestamps
view_angles
wavelengths
fwhm
dark_frame
radiometric_frame
eul_zyx
position_ecef
nav_timestamp
radiance_cube
t_exp_ms
hsi_timestamps
view_angles
wavelengths
fwhm
dark_frame
radiometric_frame
eul_zyx
position_ecef
nav_timestamp
radiance_cube
t_exp_ms
hsi_timestamps
view_angles
wavelengths
fwhm
dark_frame
radiometric_frame
eul_zyx
position_ecef
nav_timestamp
radiance_cube
t_exp_ms
hsi_timestamps
vi

AttributeError: module 'numpy' has no attribute 'matlib'

In [17]:
# We need tu unveil how to read write the data to and from storage

local_copy_config = os.path.join('/home/notebook/config_specim.ini')


# Simple example:
sb.storage.copy_file(config_template_path, local_copy_config, minio_client, overwrite=False)




OSError: Could not identify destination '/home/notebook/config_specim.ini' on MinIO.

In [ ]:
import specim_process

# Process missions
for mission_fold in mission_list:
    mission_name = mission_fold.name
    print(f"\n################\nProcessing: {mission_name}")
    conf_yaml = os.path.join(mission_fold, "config.seabee.yaml")
    specim_process.main(conf_yaml, mission_fold)